In [15]:
import scraping_class
logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)

In [11]:

import requests,os,time
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(0.5) # sleep one second.

class Connector():
    def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring. 
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessments
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case). 
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type
    
    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
            
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
    
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
     
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\n'+';'.join(map(str,row))) # write log.
          self.log.flush()
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\n'+';'.join(map(str,row))) # write row to log.
          self.log.flush()
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call. 
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\n'+';'.join(map(str,row))) # write row to log file.
      self.log.flush()
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return None,call_id

In [16]:
# SCRABE FEMALE FIRST NAMES AND OTHERS
first_female=[]
sur_female=[]
for i in range(1,8):
    url='https://imdb.com/list/ls022928836/?sort=list_order,asc&mode=detail&page={}'.format(i)
    call='Exam, KU, Female Page {}'.format(i)
    response,call_id = connector.get(url, call)
    html = response.text
    soup=str(html)
    if i < 7:
        for i in range(1,101):
            first_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))
    if i == 7:
        for i in range(1,7):
            first_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_female.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))


In [17]:
first_female

['Natalie',
 'Anne',
 'Talia',
 'Diane',
 'Keira',
 'Uma',
 'Jennifer',
 'Julianne',
 'Amy',
 'Cate',
 'Marion',
 'Helena',
 'Jessica',
 'Kate',
 'Scarlett',
 'Léa',
 'Robin',
 'Vera',
 'Tilda',
 'Gwyneth',
 'Carrie-Anne',
 'Maggie',
 'Jennifer',
 'Judi',
 'Jodie',
 'Meryl',
 'Carrie',
 'Amanda',
 'Sigourney',
 'Frances',
 'Sandra',
 'Rebecca',
 'Sally',
 'Daryl',
 'Juno',
 'Naomie',
 'Julia',
 'Emily',
 'Naomi',
 'Nicole',
 'Emma',
 'Jennifer',
 'Michelle',
 'Carey',
 'Mélanie',
 'Melissa',
 'Kathy',
 'Rachel',
 'Winona',
 'Viola',
 'Toni',
 'Emma',
 'Patricia',
 'Rosanna',
 'Ellen',
 'Catherine',
 'Elle',
 'Kerry',
 'Cameron',
 'Diane',
 'Kristen',
 'Elizabeth',
 'Laura',
 'Annette',
 'Rooney',
 'Maggie',
 'Penélope',
 'Olivia',
 'Lucy',
 'Kirsten',
 'Vivica',
 'Katie',
 'Jennifer',
 'Michelle',
 'Kelly',
 'Liv',
 'Jennifer',
 'Lea',
 'Saoirse',
 'Hilary',
 'Amy',
 'Sarah',
 'Connie',
 'Ellen',
 'Catherine',
 'Julia',
 'Rosamund',
 'Marcia',
 'Margot',
 'Marisa',
 'Franka',
 'Karen',

In [23]:
# SCRABE MALE FIRST NAMES AND OTHERS
first_male=[]
sur_male=[]
for i in range (1,8):
    url='https://www.imdb.com/list/ls022928819/?sort=list_order,asc&mode=detail&page={}'.format(i)
    call='Exam, KU, Male Page {}'.format(i)
    response,call_id = connector.get(url, call)
    html = response.text
    soup=str(html)
    if i < 7:
        for i in range(1,101):
            first_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))
    if i == 7:
        for i in range(1,82):
            first_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[1]))
            sur_male.append((soup.split('div class="lister-list"')[1].split('h3 class')[i].split('</span>\n<a href=')[1].split('>')[1].split('\n')[0].split(' ')[2:]))


[['Freeman'],
 ['DiCaprio'],
 ['Pitt'],
 ['Caine'],
 ['De', 'Niro'],
 ['Damon'],
 ['Hanks'],
 ['Bale'],
 ['Oldman'],
 ['Pacino'],
 ['Norton'],
 ['Ford'],
 ['Depp'],
 ['Murphy'],
 ['Willis'],
 ['Fiennes'],
 ['Spacey'],
 ['Nicholson'],
 ['L.', 'Jackson'],
 ['Duvall'],
 ['Seymour', 'Hoffman'],
 ['Hardy'],
 ['Buscemi'],
 ['Ruffalo'],
 ['Clooney'],
 ['Neeson'],
 ['Cruise'],
 ['Weaving'],
 ['Law'],
 ['Gyllenhaal'],
 ['Crowe'],
 ['Affleck'],
 ['Eastwood'],
 ['Keitel'],
 ['Gordon-Levitt'],
 ['Gosling'],
 ['Wilkinson'],
 ['Harrelson'],
 ['Cooper'],
 ['Washington'],
 ['Robbins'],
 ['Kingsley'],
 ['Hurt'],
 ['Cazale'],
 ['Carroll', 'Lynch'],
 ['Harris'],
 ['Affleck'],
 ['McConaughey'],
 ['Waltz'],
 ['Roth'],
 ['Ledger'],
 ['Murray'],
 ['Fassbender'],
 ['Postlethwaite'],
 ['Madsen'],
 ['Pepper'],
 ['Ejiofor'],
 ['Rickman'],
 ['Pearce'],
 ['Foxx'],
 ['Rush'],
 ['Giamatti'],
 ['Bloom'],
 ['Sheen'],
 ['Goodman'],
 ['Lloyd'],
 ['Baldwin'],
 ['Hoffman'],
 ['Williamson'],
 ['Pesci'],
 ['Hawke'],
 ['Chea

In [24]:
import nltk
name=nltk.corpus.names
# CHECKING FOR OVERLAP IN THE ORIGINAL LIST
name.fileids()
name.words('male.txt') in name.words('female.txt')
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3
check=intersection(first_f,name.words('female.txt'))
# REMOVING DUPLICATES BY MAKING INTO DICT AND BACK AGAIN
first_m=list(dict.fromkeys(first_male))
first_f=list(dict.fromkeys(first_female))

male_name=name.words('male.txt')+first_m
len(male_name)
first_m_done=list(dict.fromkeys(male_name))
len(first_m_done)

female_name=name.words('female.txt')+first_f
print(len(female_name))
first_f_done=list(dict.fromkeys(female_name))
print(len(first_f_done))
print(len(name.words('female.txt')))

NameError: name 'first_f' is not defined

In [28]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page=1&title_type=movie&fbclid=IwAR3Gom1la1nQpTDUc_I3UbRpkAud67ieRBaVYkreIXPeMyoSua_LZT2sFrs&release_date=1980%2C2019&sort=num_votes%2Cdesc'.format(j)
response,callid = connector.get(url,'Exam')
html = response.text
soup = BeautifulSoup(html,'lxml')

#.findAll("span", {"name": "nv"})[1]

In [29]:
# BOX OFFICE DATA
Box=[]
for j in range (1,2):
    url = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page=1&title_type=movie&fbclid=IwAR3Gom1la1nQpTDUc_I3UbRpkAud67ieRBaVYkreIXPeMyoSua_LZT2sFrs&release_date=1980%2C2019&sort=num_votes%2Cdesc'.format(j)
    response,callid = connector.get(url,'Exam')
    html = response.text
    soup = BeautifulSoup(html,'lxml')
    for i in range(50):
        if len(soup.findAll("div", {"class": "lister-item mode-detail"})[i].findAll("span", {"name": "nv"})) < 2:
            Box.append(None)
        if len(soup.findAll("div", {"class": "lister-item mode-detail"})[i].findAll("span", {"name": "nv"})) == 2:
            Box.append(soup.findAll("div", {"class": "lister-item mode-detail"})[i].findAll("span", {"name": "nv"})[1].text)


In [30]:
# SCRABE STAR-RATINGS (IMDB)
rating=[]
for j in range (1,201):
    url = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page=1&title_type=movie&fbclid=IwAR3Gom1la1nQpTDUc_I3UbRpkAud67ieRBaVYkreIXPeMyoSua_LZT2sFrs&release_date=1980%2C2019&sort=num_votes%2Cdesc'.format(j)
    response,callid = connector.get(url,'Exam')
    html = response.text
    soup = BeautifulSoup(html,'lxml')
    print(j)
    for i in range(50):
        if soup.findAll("div", {"class": "lister-item mode-detail"})[i].findAll("div", {"class": "inline-block ratings-imdb-rating"})==[]:
            rating.append(None)
        else:
            primo=soup.findAll("div", {"class": "lister-item mode-detail"})[i].findAll("div", {"class": "inline-block ratings-imdb-rating"})[0].text
            rating.append(primo.split('\n')[2])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [31]:
# SCRABE DIRECTOR NAMES
director=[]
for j in range (1,201):
    url = 'https://www.imdb.com/search/keyword/?ref_=kw_ref_yr&mode=detail&page={}&title_type=movie&fbclid=IwAR3Gom1la1nQpTDUc_I3UbRpkAud67ieRBaVYkreIXPeMyoSua_LZT2sFrs&release_date=1980%2C2019&sort=num_votes%2Cdesc'.format(j)
    response,callid = connector.get(url,'Exam')
    html = response.text
    soup = BeautifulSoup(html,'lxml')


    for i in range(50):
        primo=soup.findAll("div", {"class": "lister-item mode-detail"})[i].findAll("p", {"class": "text-muted text-small"})[1].text
        if len(primo.split('Stars')[0].split('\n')) > 5:
            director.append(primo.split('Stars')[0].split('\n')[2:4])#+primo.split('Stars')[0].split('\n')[3])
        else:
            director.append(primo.split('Stars')[0].split('\n')[2])
        if len(primo.split('Stars')[0].split('\n')) > 6:
            print(len(primo.split('Stars')[0].split('\n')))

7
7
7
7
7
7
7
7
8
7
7
9
7
7
7
7
7
17
8
7
7
7
7
26
7
14
7
8
7
7
15
7
8
8
10
7
11
7
7
12
7
8
14
7
7
7
7
7
7
31
7
8
7
7
7
7
7
12
38
7
7
10
7
8
8
7
7
7
7
7
15
7
7
8
8
9
7
13
7
7
8
7
7
9
9
8
7
35
12
7
7
8
7
7
10
7
7
9
7
7
9
7
7
8
7
7
8


In [33]:
len(director)

10000

In [34]:
director

['Frank Darabont',
 'Christopher Nolan',
 'Christopher Nolan',
 'David Fincher',
 'Quentin Tarantino',
 'Robert Zemeckis',
 ['Lana Wachowski, ', 'Lilly Wachowski'],
 'Peter Jackson',
 'Peter Jackson',
 'Christopher Nolan',
 'Peter Jackson',
 'Christopher Nolan',
 'David Fincher',
 'Quentin Tarantino',
 'Ridley Scott',
 'Christopher Nolan',
 'Joss Whedon',
 'Jonathan Demme',
 'Quentin Tarantino',
 'Steven Spielberg',
 'Steven Spielberg',
 'Martin Scorsese',
 'Christopher Nolan',
 'Irvin Kershner',
 'James Cameron',
 'Martin Scorsese',
 'Christopher Nolan',
 'Frank Darabont',
 'Martin Scorsese',
 'Sam Mendes',
 'Gore Verbinski',
 'James Cameron',
 'James Gunn',
 'Tony Kaye',
 'James McTeigue',
 'Robert Zemeckis',
 'Luc Besson',
 'James Cameron',
 'Martin Scorsese',
 'Andrew Stanton',
 'Quentin Tarantino',
 'Bryan Singer',
 'Mel Gibson',
 ['Andrew Stanton, ', 'Lee Unkrich'],
 'Jon Favreau',
 'Richard Marquand',
 ['Pete Docter, ', 'Bob Peterson'],
 ['Roger Allers, ', 'Rob Minkoff'],
 'M. N